### 1. Formulate the hypothesis

Hypotheses:

Ho:  p = (better than) Po

Ha: p (not better than) Po


Where:

Ho: The new home page design performs better than the old design

Ha: The new home page design does not perform better than the old design

p: 55%

Po: 50%

### 2. Conduct power analysis

In [2]:
# [1] Import the ‘statsmodel’ for statistical calculations and
# [1a] the ‘TTestIndPower’ class to calculate the parameters for the analysis:
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower


# [2] Specify the three required parameters for the power analysis:
# [2a] Specify ‘alpha’
alpha = 0.05 
# [2b] Specify ‘power’
power = 0.8 
# [2c] Specify ‘effect’ and calculate the minimum effect.
effect = sms.proportion_effectsize(0.50, 0.55) 


# [3] Perform power analysis by using the solve_power() function:
# [3a] Specify an instance of ‘TTestIndPower’
analysis = TTestIndPower() 

# [3b] Calculate the sample size and list the parameters
result = analysis.solve_power(effect, power=power, nobs1=None,
                              ratio=1.0, alpha=alpha) 

# [4] Print the output up to 3 decimal places (with lead-in text)
print('Sample Size: %.3f' % result)

Sample Size: 1565.490


### 3. Import packages and Load data

In [3]:
# Install the relevant modules:
!pip install scipy

# Import necessary libraries, packages and classes
import pandas as pd
# Import Python's built-in maths module
import math
import numpy as np
# Import statsmodels stats test and tools
import statsmodels.stats.api as sms
# Import the scipy.stats for more stats functions
import scipy.stats as st
# Import Matplotlib for visualisation tools
import matplotlib as mpl
import matplotlib.pyplot as plt

In [21]:
# Read the 'ab.data.csv' file
df = pd.read_csv('new_bike_shop_AB.csv')

# View the DataFrame
df.head()

Unnamed: 0  RecordID    IPAddress  LoyaltyPage  ServerID  VisitPageFlag
0           0         1  39.13.114.2            1         2              0
1           1         2    13.3.25.8            1         1              0
2           2         3  247.8.211.8            1         1              0
3           3         4  124.8.220.3            0         3              0
4           4         5  60.10.192.7            0         2              0

In [22]:
# Check the metadata for information about the file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184588 entries, 0 to 184587
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     184588 non-null  int64 
 1   RecordID       184588 non-null  int64 
 2   IPAddress      184588 non-null  object
 3   LoyaltyPage    184588 non-null  int64 
 4   ServerID       184588 non-null  int64 
 5   VisitPageFlag  184588 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 8.4+ MB


In [23]:
# rename columns
df_new = df.rename(columns={'IP Address':'IPAddress','LoggedInFlag':'LoyaltyPage'})

In [9]:
# Use Pandas's duplicated() function to check the user_id column 
print(df_new[df.IPAddress.duplicated()])


        Unnamed: 0  RecordID     IPAddress  LoyaltyPage  ServerID  \
275            275       276    191.4.97.7            0         2   
394            394       395     79.9.70.7            1         3   
703            703       704    175.1.81.8            1         3   
809            809       810    125.0.30.9            1         2   
889            889       890  207.14.157.6            1         3   
...            ...       ...           ...          ...       ...   
184582      184582    184583    90.4.224.4            0         3   
184583      184583    184584   114.8.104.1            0         1   
184585      184585    184586   170.13.31.9            0         2   
184586      184586    184587   195.14.92.3            0         3   
184587      184587    184588  172.12.115.8            0         2   

        VisitPageFlag  
275                 0  
394                 0  
703                 0  
809                 0  
889                 0  
...               ...  
184

In [24]:
# Use ‘drop_duplicates’ to return the Series without the duplicate values
df_new = df.drop_duplicates(subset = "IPAddress") 

df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99516 entries, 0 to 184584
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     99516 non-null  int64 
 1   RecordID       99516 non-null  int64 
 2   IPAddress      99516 non-null  object
 3   LoyaltyPage    99516 non-null  int64 
 4   ServerID       99516 non-null  int64 
 5   VisitPageFlag  99516 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 5.3+ MB


In [25]:
# Use ‘dropped.drop’ to remove irrelevant columns from the DataFrame 
# and specify the column names:
# Specify that ‘user_id’ and ‘timestamp’ are columns (i.e. ‘axis 1’) 
df_new = df_new.drop(["Unnamed: 0", "RecordID",'VisitPageFlag'], axis=1)  

df_new.head()

IPAddress  LoyaltyPage  ServerID
0  39.13.114.2            1         2
1    13.3.25.8            1         1
2  247.8.211.8            1         1
3  124.8.220.3            0         3
4  60.10.192.7            0         2

In [29]:
#df_new['ServerID'] = df_new.ServerID.map({1:'Treatment', 2:'Control', 3:'Control'})

df_new.head()


IPAddress  LoyaltyPage   ServerID
0  39.13.114.2            1    Control
1    13.3.25.8            1  Treatment
2  247.8.211.8            1  Treatment
3  124.8.220.3            0    Control
4  60.10.192.7            0    Control

In [36]:
# Obtain a simple random sample for control and treatment groups with n = 4721; 
# set random_stategenerator seed at an arbitrary value of 22.

# Obtain a simple random sample for the control group.
control_sample = df_new[df_new['ServerID']=='Control'].sample(n=1566,
                                                                   random_state=22) 
# Obtain a simple random sample for the treatment group.
treatment_sample = df_new[df_new['ServerID']=='Treatment'].sample(n=1566,
                                                                       random_state=22)

In [38]:
control_sample.head()

IPAddress  LoyaltyPage ServerID
68046      161.2.4.2            1  Control
184269     226.5.5.1            0  Control
109050  144.11.220.8            1  Control
3492     253.7.255.7            0  Control
12188       4.4.81.2            0  Control

In [39]:
# [1] Join the two samples.  
ab_test = pd.concat([control_sample, treatment_sample], axis=0)  

# [2] Reset the A/B index.
ab_test.reset_index(drop=True, inplace=True) 

# [3] Print the sample table.
ab_test  

IPAddress  LoyaltyPage   ServerID
0        161.2.4.2            1    Control
1        226.5.5.1            0    Control
2     144.11.220.8            1    Control
3      253.7.255.7            0    Control
4         4.4.81.2            0    Control
...            ...          ...        ...
3127    36.3.109.5            1  Treatment
3128   187.14.60.2            0  Treatment
3129    112.16.0.7            1  Treatment
3130    18.13.42.9            1  Treatment
3131   132.0.110.9            0  Treatment

[3132 rows x 3 columns]

In [42]:
# [1] import library
from scipy.stats import sem

# [2] Group the ab_test data set by group and aggregate by converted:
conversion_rates = ab_test.groupby('ServerID')['LoyaltyPage']


# [3] Calculate conversion rates by calculating the means of columns STD_p and SE_p:
conversion_rates = conversion_rates.agg([np.mean, np.std,sem])

# [4] Assign names to the three columns.
conversion_rates.columns = ['conversion_rate', 'std_deviation','std_error']  

 # [5] Round the output to 3 decimal places
conversion_rates.style.format('{:.3f}') 

In [43]:
# [1] Import proportions_ztest and proportion_confint from statsmodels:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# [2] Create a subset of control and treatment results:
control_results = ab_test[ab_test['ServerID'] == 'Control']['LoyaltyPage']
treatment_results = ab_test[ab_test['ServerID'] == 'Treatment']['LoyaltyPage']

# [3] Determine the count of the control_results and 
# treatment_result sub-datasets and store them in their respective variables:
n_con = control_results.count()
n_treat = treatment_results.count()

# [4a] Create a variable ‘success’ with the sum of the two data sets
# in a list format: 
successes = [control_results.sum(), treatment_results.sum()]

# [4b] Create a variable ‘nobs’ which stores the values of 
# variables n_con and n_treat in list format: 
nobs = [n_con, n_treat] 

# [5] Use the imported libraries to calculate the statistical values: 
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes,
                                                                        nobs=nobs,
                                                                        alpha=0.05)

# [6a-d] Print the outputs (with lead-in text):
print(f'Z test stat: {z_stat:.2f}')
print(f'P-value: {pval:.3f}')
print(f'Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

Z test stat: 0.07
P-value: 0.943
Confidence Interval of 95% for control group: [0.478, 0.527]
Confidence Interval of 95% for treatment group: [0.477, 0.526]
